---
title: "Practice Activity 4.1: Webscraping"
author: "Shiqi Wu"
format:
  html:
    self-contained: true
jupyter: python3
execute:
  echo: true
---

**Repository (Week 4 – Activity 4.1):**  [Week 4](https://github.com/shiqiwu212/GSB-S544-01/tree/2a937d3e40d7ff250b14fa41e7dd0212dae4ef5f/Week%204/Practice%20Activities/Practice%20Activity%204.1)  




# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [18]:
import pandas as pd

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [19]:
import requests

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
HEADERS  = {"User-Agent": "Mozilla/5.0 (compatible; MSBA-Student/1.0)"}

resp = requests.get(WIKI_URL, headers=HEADERS, timeout=20)
resp.raise_for_status()
html_text = resp.text

Use Beautiful Soup to parse this string into a tree called `soup`

In [20]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_text, "html.parser")

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [21]:
len(soup.find_all("table"))

10

We can use attributes like `class=` and `style=` to narrow down the list.

In [22]:
len(soup.find_all("table",
                  attrs={
                      "class": "wikitable sortable",
                      "style": "text-align:center"}
                  ))

0

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [23]:
# AI assistance for this step:
# - Keep the instructor's selection; add minimal fallbacks so the cell won't crash if no exact match.
try:
    table = soup.find_all("table",
                          attrs={"class": "wikitable sortable",
                                 "style": "text-align:center"})[0]  # ← instructor line
except IndexError:
    # Fallback 1: class-only (common on Wikipedia)
    matches = soup.find_all("table", class_="wikitable sortable")
    if matches:
        table = matches[0]
    else:
        # Fallback 2: pick the largest table by number of rows
        all_tbls = soup.find_all("table")
        if not all_tbls:
            raise RuntimeError("No <table> elements found; check Steps 1–2 and your network.")
        table = max(all_tbls, key=lambda t: len(t.find_all("tr")))

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [24]:
# AI assistance for this step:
# - Minimal, safe parse: take header row, iterate data rows, and align with headers.
import re

def _clean(text: str) -> str:
    text = re.sub(r"\[[^\]]*\]", "", text)      # drop footnote markers like [a], [note 1]
    return " ".join(text.split())

# header
header_row   = table.find("tr")
header_cells = header_row.find_all(["th", "td"]) if header_row else []
headers      = [_clean(h.get_text(strip=True)) for h in header_cells]

# rows
rows = []
for tr in table.find_all("tr")[1:]:
    tds = tr.find_all("td")
    if not tds:
        continue
    vals = [_clean(td.get_text(strip=True)) for td in tds]
    # align to header length (pad/trim) to avoid shape mismatches
    if len(headers):
        if len(vals) < len(headers):
            vals += [""] * (len(headers) - len(vals))
        elif len(vals) > len(headers):
            vals = vals[:len(headers)]
        rows.append(dict(zip(headers, vals)))
    else:
        # if no headers, just collect raw cells
        rows.append({"col_"+str(i): v for i, v in enumerate(vals)})

df_cities_bs = pd.DataFrame(rows)
df_cities_bs.head()

,City,ST,2024estimate,2020census,Change,2020 land area,2020 density,Location
0,New York,NY,"8,478,072","8,804,190",−3.70%,300.5,778.3,"29,298"
1,Los Angeles,CA,"3,878,704","3,898,747",−0.51%,469.5,"1,216.0","8,304"
2,Chicago,IL,"2,721,308","2,746,388",−0.91%,227.7,589.7,"12,061"
3,Houston,TX,"2,390,125","2,304,580",+3.71%,640.4,"1,658.6","3,599"
4,Phoenix,AZ,"1,673,164","1,608,139",+4.04%,518.0,"1,341.6","3,105"


Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [25]:
# AI assistance for this step:
# Fixed blocked (SSL/403), use the HTML we already fetched.

from io import StringIO

try:
    pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")  
except Exception:
    pd.read_html(StringIO(html_text))  # same effect, avoids SSL/403 by not re-fetching

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [26]:
# AI assistance for this step:
# - If none are found (Wikipedia sometimes drops the 'style'), minimally relax to class-only.

# strict attrs 
try:
    tables_attr = pd.read_html(
        StringIO(html_text),
        attrs={'class': 'wikitable sortable', 'style': 'text-align:center'}
    )
except ValueError:
    tables_attr = []

# minimal relaxation: class-only
if not tables_attr:
    try:
        tables_attr = pd.read_html(
            StringIO(html_text),
            attrs={'class': 'wikitable sortable'}
        )
    except ValueError:
        tables_attr = []

# last resort: parse a single matching <table> fragment so the step still demonstrates the filter idea
if not tables_attr:
    s5 = BeautifulSoup(html_text, "html.parser")
    tbl = s5.select_one("table.wikitable.sortable") or s5.select_one("table.wikitable")
    tables_attr = pd.read_html(StringIO(str(tbl))) if tbl else []

tables_attr  

[            City  ST 2024 estimate 2020 census  Change 2020 land area          \
             City  ST 2024 estimate 2020 census  Change            mi2     km2   
 0    New York[c]  NY       8478072     8804190  −3.70%          300.5   778.3   
 1    Los Angeles  CA       3878704     3898747  −0.51%          469.5  1216.0   
 2        Chicago  IL       2721308     2746388  −0.91%          227.7   589.7   
 3        Houston  TX       2390125     2304580  +3.71%          640.4  1658.6   
 4        Phoenix  AZ       1673164     1608139  +4.04%          518.0  1341.6   
 ..           ...  ..           ...         ...     ...            ...     ...   
 341      Deltona  FL        100513       93692  +7.28%           37.3    96.6   
 342  Federal Way  WA        100252      101030  −0.77%           22.3    57.8   
 343   San Angelo  TX        100159       99893  +0.27%           59.7   154.6   
 344        Tracy  CA        100136       93000  +7.67%           25.9    67.1   
 345      Sunris

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [27]:
# AI assistance for this step:
# - Keep the "first match" behavior; fall back to the first page-wide table,
#   or (if needed) the first <table> fragment to ensure a DataFrame is produced.

if tables_attr:
    df_cities2 = tables_attr[0]
elif tables_all:
    df_cities2 = tables_all[0]
else:
    s53 = BeautifulSoup(html_text, "html.parser")
    frag = s53.find("table")
    df_cities2 = pd.read_html(StringIO(str(frag)))[0] if frag else pd.DataFrame()

df_cities2.head()

City  ST 2024 estimate 2020 census  Change 2020 land area          \
          City  ST 2024 estimate 2020 census  Change            mi2     km2   
0  New York[c]  NY       8478072     8804190  −3.70%          300.5   778.3   
1  Los Angeles  CA       3878704     3898747  −0.51%          469.5  1216.0   
2      Chicago  IL       2721308     2746388  −0.91%          227.7   589.7   
3      Houston  TX       2390125     2304580  +3.71%          640.4  1658.6   
4      Phoenix  AZ       1673164     1608139  +4.04%          518.0  1341.6   

  2020 density                                      Location  
         / mi2  / km2                               Location  
0        29298  11312    40°40′N 73°56′W﻿ / ﻿40.66°N 73.94°W  
1         8304   3206  34°01′N 118°25′W﻿ / ﻿34.02°N 118.41°W  
2        12061   4657    41°50′N 87°41′W﻿ / ﻿41.84°N 87.68°W  
3         3599   1390    29°47′N 95°23′W﻿ / ﻿29.79°N 95.39°W  
4         3105   1199  33°34′N 112°05′W﻿ / ﻿33.57°N 112.09°W

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [28]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [29]:
# AI assistance for this step:
# - Tolerant extraction for "Term Typically Offered" (handles variants and optional colon).

import re  

try:
    cat_html = cat_resp.text  
except NameError:
    CATALOG_URL = globals().get(
        "CATALOG_URL",
        "http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory"
    )
    r = requests.get(CATALOG_URL, timeout=20)
    r.raise_for_status()
    cat_html = r.text

soup2 = BeautifulSoup(cat_html, "html.parser")

records = []
for block in soup2.find_all("div", class_="courseblock"):
    title_tag = block.find("p", class_="courseblocktitle")
    if not title_tag:
        continue
    course_title = title_tag.get_text(" ", strip=True)

    # patched term extraction: accept "Term Typically Offered" or "Typically Offered", optional colon
    term = ""
    for p in block.find_all("p", class_="noindent courseblockextra"):
        txt = p.get_text(" ", strip=True)
        m = re.search(r"(?:Term\s+Typically\s+Offered|Typically\s+Offered)\s*:?\s*(.*)", txt, flags=re.I)
        if m:
            term = m.group(1)
            break

    records.append({"Course": course_title, "Term_Typically_Offered": term})

df_courses = pd.DataFrame(records)
df_courses.head()

,Course,Term_Typically_Offered
0,DATA 100. Data Science for All I. 4 units,
1,DATA 301. Introduction to Data Science. 4 units,
2,DATA 401. Data Science Process and Ethics. 3 u...,
3,DATA 402. Mathematical Foundations of Data Sci...,
4,DATA 403. Data Science Projects Laboratory. 1 ...,


Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag